# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
# Define the function priest() with the inputs described
def priest(sex, age_yrs, respRate, o2sat, HR_bpm, sysBP_mmHg, temp_c, alert, inspO2, performance):
    '''(str, int, int, float, int, int, float, str, str, str) -> float
    This function calculates a PRIEST clinical severity score using the algorithm described (https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence)
    and returns the 30-day probability of adverse outcome as a decimal, where any raw score from 17-25 returns 0.59 and any raw score >=26 returns 0.99. 
    The inputs are as follows:
        sex - male or female (or any permutation thereof: Male, fEMale, m, f, M, F, etc.)
        age_yrs - Age in years (16 years and older, with a check for age > 120 years)
        respRate - Respiratory rate in breaths per minute
        o2sat - Oxygen saturation as a percent between 0 and 1
        HR_bpm - Heart rate in beats per minute
        sysBP_mmHg - Systolic BP in mmHg
        temp_c - Temperature in degrees C
        alert - Alertness as a string description ("alert", "confused", "not alert")
        inspO2 - Inspired Oxygen as as string description ("air", "supplemental oyxgen")
        performance - Performance Status as a string description ("unrestricted normal activity", "limited strenous activity", "limited activity", "limited self-care"
        "bed/chair bound")
    If any input is invalid, a ValueException is raised with a description of the invalid input and direction to the help documentation. 

    >>> priest("female", 24, 22, 0.96, 100, 94, 35.5, "alert", "air", "Unrestricted normal activity")
    0.15
    
    >>> priest("female", 24, 22, 0.96, 100, 94, 35.5, "alert", "air", "bed/chair bound, no self-care")
    0.29
    
    >>> priest("Female", 24, 22, 0.96, 100, 94, 35.5, "Alert", "aiR", "bed/chair bound, no self-care")
    0.29
    
    >>> priest("Male", 24, 22, 0.96, 100, 94, 35.5, "Alert", "aiR", "Limited self-care")
    0.29
    '''
    
    # Create a variable for the raw score - if -1 is returned, function did not compute score
    raw = -1
    
    # Create a variable for the percentile/30-day probability
    adverse_probability = -1
    
    # Assign scores for each variable based on the algorithm provided    
    # change the input string to all lowercase and determine male or female
    # Assign 0 points for female and 1 point for male
    if sex.lower()== "male" or sex.startswith("m") or sex.startswith("M"):
        raw = 1
    elif sex.lower() == "female" or sex.startswith("f") or sex.startswith("F"):
        raw = 0
    else:
        raise ValueError("""The input for sex was not recognized as male or female. 
        The input provided was {}.
        Please review the help documentation for acceptable inputs.""".format(sex))
    
    # Assign points based on age range (16-49: 0; 50-65: 2; 66-80: 3; >80: 4)
    if (age_yrs >= 16) and (age_yrs <= 49):
        raw += 0
    elif (age_yrs >= 50) and (age_yrs <= 65):
        raw += 2
    elif(age_yrs >= 66) and (age_yrs <=80):
        raw += 3
    elif (age_yrs > 80):
        raw += 4
    # Any age less than 16 (not a valid input) or greater than 120 (extremely unlikely input
    # or any other input not recognized by the above rules raises an exception
    elif (age < 16):
        raise ValueError("""The input for age in years is less than 16. The PRIEST score
        is only calculated for patients ages 16 years or older.
        The input provided was {}.""".format(age_yrs)) 
    elif (age > 120):
        raise ValueError("""The input for age is greater than 120. Please ensure you are entering
        age in years. The input provided was {}.""".format(age_yrs))
    else:
        raise ValueError("""The input for age in years was invalid.
        The input provided was {}.
        Please review the help documentation for acceptable inputs.""".format(age_yrs))
        
    # Assign points based on the respiratory rate in breaths per minute
    if respRate < 9:
        raw += 3
    elif (respRate >= 9) and (respRate <= 11):
        raw += 1
    elif (respRate >= 12) and (respRate <=20):
        raw += 0
    elif (respRate >=21) and (respRate <= 24):
        raw += 2
    elif (respRate > 24):
        raw += 3
    else:
        raise ValueError("""The input for respiratory rate (breaths per minute) was invalid. 
        The input provided was {}.
        Please review the help documentation for acceptable inputs.""".format(respRate)) 
    
    # Assign points based on the oxygen saturation (value from 0 -1)
    if o2sat > 0.95:
        raw += 0
    elif (o2sat <= 0.95) and (o2sat >=0.94):
        raw += 1
    elif (o2sat < 0.94) and (o2sat >=0.92):
        raw +=2
    elif (o2sat < 0.92):
        raw += 3
    else:
        raise ValueError("""The input for oxygen saturation was invalid. The percent value 
        should be converted to a decimal between 0 and 1. 
        The input provided was {}. 
        Please review the help documentation for acceptable inputs.""".format(o2sat))         
    
    # Assign points based on heart rate in beats per minute
    if HR_bpm < 41:
        raw += 3
    elif (HR_bpm >= 41) and (HR_bpm <= 50):
        raw += 1
    elif (HR_bpm >= 51) and (HR_bpm <= 90):
        raw += 0
    elif(HR_bpm >= 91) and (HR_bpm <= 110):
        raw += 1
    elif(HR_bpm >= 111) and (HR_bpm <= 130):
        raw += 2
    elif (HR_bpm > 130):
        raw += 3
    else:
        raise ValueError("""The input for heart rate (beats per minute) was invalid. 
        The input provided was {}.
        Please review the help documentation for acceptable inputs.""".format(HR_bpm))   
    
    # Assign points based on systolic BP in mmHg
    if (sysBP_mmHg < 91):
        raw += 3
    elif (sysBP_mmHg >= 91) and (sysBP_mmHg <=100):
        raw += 2
    elif (sysBP_mmHg >=101) and (sysBP_mmHg <= 110):
        raw += 1
    elif(sysBP_mmHg >= 111) and (sysBP_mmHg <=219):
        raw += 0
    elif (sysBP_mmHg >= 219):
        raw += 3
    else:
        raise ValueError("""The input for systolic blood pressure (mmHg) was invalid. 
        The input provided was {}.
        Please review the help documentation for acceptable inputs.""".format(sysBP_mmHg))  
    
    # Assign points based on temperature in degrees Celsius
    # Check for an accidental input in Farenheit - anything over 50 is considered suspect
    if (temp_c >=50):
        raise ValueError("""The input for temperature (degrees Celsius) was greater than 
        or equal to 50. The input provided was {}. Be sure to input the temperature in Celsius.
        Please review the help documentation for acceptable inputs.""". format(temp_c))
    elif (temp_c < 35.1):
        raw += 3
    elif (temp_c >=35.1) and (temp_c <=36.0):
        raw += 1
    elif (temp_c >= 36.1) and (temp_c <= 38.0):
        raw += 0
    elif (temp_c >= 38.1) and (temp_c <= 39.0):
        raw += 1
    elif (temp_c > 39):
        raw += 2
    else:
        raise ValueError("""The input for temperature (degrees Celsius) was invalid. 
        The input provided was {}. Be sure to input the temperature in Celsius.
        Please review the help documentation for acceptable inputs.""". format(temp_c))
    
    # Assign points based on alertness
    # Ensure the input string is all lowercase prior to comparing to standard values
    if (alert.lower() == "alert"):
        raw += 0
    elif ((alert.lower()) == "confused or not alert"):
        raw += 3
    else:
        raise ValueError("""The input for alertness was invalid. The input provided was {}. 
        Please review the help documentation for acceptable inputs.""". format(alert))
   
    # Assign points based on source of inspired oxygen
    # Ensure the input string is all lowercase prior to comparing to standard values
    if inspO2.lower() == "air":
        raw += 0
    elif inspO2.lower() == "supplemental oxygen":
        raw += 2
    else:
        raise ValueError("""The input for inspired oxygen was invalid. The input provided was {}. 
        Please review the help documentation for acceptable inputs.""". format(inspO2))
    
    # Assign points based on performance status
    # Ensure the input string is all lowercase prior to comparing to standard values
    if performance.lower() == "unrestricted normal activity":
        raw += 0
    elif performance.lower() == "limited strenuous activity, can do light activity":
        raw += 1
    elif performance.lower() == "limited activity, can self-care":
        raw += 2
    elif performance.lower() == "limited self-care":
        raw += 3
    elif performance.lower() == "bed/chair bound, no self-care":
        raw += 4
    else:
        raise ValueError("""The input for performance status was invalid. The input provided was {}. 
        Please review the help documentation for acceptable inputs.""". format(performance))

    # Based on the calculated raw score, determine the 30-day probability of adverse outcome
    if (raw > 0) and (raw <= 1):
        adverse_probability = 0.01
    elif (raw >= 2) and (raw <= 3):
        adverse_probability = 0.02
    elif (raw == 4):
        adverse_probability = 0.03
    elif (raw == 5):
        adverse_probability = 0.09
    elif (raw == 6):
        adverse_probability = 0.15
    elif (raw == 7):
        adverse_probability = 0.18
    elif (raw == 8):
        adverse_probability = 0.22
    elif (raw == 9):
        adverse_probability = 0.26
    elif (raw == 10):
        adverse_probability = 0.29
    elif (raw == 11):
        adverse_probability = 0.34
    elif (raw == 12):
        adverse_probability = 0.38
    elif (raw == 13):
        adverse_probability = 0.46
    elif (raw == 14):
        adverse_probability = 0.47
    elif (raw == 15):
        adverse_probability = 0.49
    elif (raw == 16):
        adverse_probability = 0.55
    elif (raw >= 17) and (raw <= 25):
        adverse_probability = 0.59
    elif (raw>=26):
        adverse_probability = 0.99
    else:
        raise ValueError("""The raw score calculated was {}. However, no adverse
        score could be calculated. Please consult the help documentation and check all inputs.""".format(raw))
    
    # Return the final score
    return adverse_probability

In [2]:
# Testing the docstring
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest("female", 24, 22, 0.96, 100, 94, 35.5, "alert", "air", "Unrestricted normal activity")
Expecting:
    0.15
ok
Trying:
    priest("female", 24, 22, 0.96, 100, 94, 35.5, "alert", "air", "bed/chair bound, no self-care")
Expecting:
    0.29
ok
Trying:
    priest("Female", 24, 22, 0.96, 100, 94, 35.5, "Alert", "aiR", "bed/chair bound, no self-care")
Expecting:
    0.29
ok
Trying:
    priest("Male", 24, 22, 0.96, 100, 94, 35.5, "Alert", "aiR", "Limited self-care")
Expecting:
    0.29
ok


In [3]:
# Testing the ValueErrors - sex
#priest("puppet", 24, 22, 0.96, 100, 94, 35.5, "alert", "air", "Unrestricted normal activity")

In [4]:
# Testing the ValueErrors - temperature
#priest("female", 24, 22, 0.96, 100, 94, 98, "alert", "air", "Unrestricted normal activity")

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [5]:
# Define the function find_hospital with the parameters above
def find_hospital(age, sex, risk):
    '''(int, str, float) -> str
    This function takes several parameters, querys a REST web service based on these parameters to determine the appropriate hospital for this patient,
    and returns the name of the hospital as a string. The information returned from the REST web service is in the form of a json file formatted as follows:
        {
          "age": "40",
          "sex": "male",
          "risk": "0.1",
          "hospital": "Southwest Hospital and Medical Center"
        }
        
    >>> find_hospital(40, "male", 0.1)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(40, "female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'
    
    >>> find_hospital(24, "female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'
    
    >>> find_hospital(40, "Male", 0.2)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(80, "Male", 0.59)
    'Wesley Woods Geriatric Hospital'
    
    >>> find_hospital(80, "female", 0.59)
    'Wesley Woods Geriatric Hospital'
    '''
    # Initialize the return variable hospital to None - if anything fails, None should return
    hospital = None
    
    # Import the JSON library
    import json
    
    # Import the requests library
    import requests
    
    # Create the appropriate URL based on the inputs
    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={}&sex={}&risk_pct={}".format(age, sex, risk)
    
    # Request the json file using the appropriate URL
    response = requests.get(url)
    
    # Extract the data into the JSON file
    data = response.json()
    
    # Locate the hospital using the key "hospital"
    hospital = data["hospital"]
    
    # Return the hospital name
    return hospital

In [6]:
# Testing different parameters to create minimum number of test cases
find_hospital(40, "female", 0.15)

'Select Specialty Hospital - Northeast Atlanta'

In [7]:
find_hospital(24, "female", 0.15)

'Select Specialty Hospital - Northeast Atlanta'

In [8]:
find_hospital(40, "Male", 0.2)

'Southwest Hospital and Medical Center'

In [9]:
find_hospital(80, "Male", 0.59)

'Wesley Woods Geriatric Hospital'

In [10]:
find_hospital(80, "female", 0.59)

'Wesley Woods Geriatric Hospital'

In [11]:
# Test the docstring 
doctest.run_docstring_examples(find_hospital, globals(), verbose = True)

Finding tests in NoName
Trying:
    find_hospital(40, "male", 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40, "female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(24, "female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(40, "Male", 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(80, "Male", 0.59)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(80, "female", 0.59)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [12]:
# Define the function as specified
def get_address(hospital):
    """(str) -> str
    This function takes a hospital name as an input. This function assumes the hospital is in Atlanta, Georgia, and that
    the address information is listed on this website: https://www.officialusa.com/stateguides/health/hospitals/georgia.html.
    
    If the hospital name is not found or there is more than one hospital with the same name, a KeyError will be raised.
    
    >>> get_address('Select Specialty Hospital - Northeast Atlanta')
    '1821 CLIFTON ROAD NE'
    
    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'
    
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    """
    # Import requests to facilitate url request
    import requests
    
    # Import Beautiful Soup to parse HTML
    from bs4 import BeautifulSoup
    
    # Initialize a variable for the address string - set to None for a final check
    address = None
    
    # Intialize a dictionary to contain the information from the website, with format:
    # hospital name as key, hospital address as value
    directory = {}
    
    # Retrieve the information from the url
    response = requests.get("https://www.officialusa.com/stateguides/health/hospitals/georgia.html")
    
    # Check for a 200 (successful) status code - if successful, retrieve the data from the response
    if response.status_code == 200:
        soup = BeautifulSoup(response.text)
    # If any code other than 200, raise a ValueError
    else:
        raise ValueError("The source data could not be retrieved from the URL. Please review help documentation.")
    
    # Create a directory by parsing the HTML
    # The hospitals and addresses are stored in a table, under the body, with each row containing information including
    # the city, hospital name, and type
    # Loop through the rows in the table
    for row in soup.find_all("tr"):
        # retrieve the text (x.text) for each element in the list created by row.find_all
        data = [x.text for x in list(row.find_all('td'))]

        # If the row has table data tags, data will not be empty
        # If the row has no table data tags, go on to the next row without examining the row contents:
        if data != []:
            # If the first element (index 0) - city name - is "ATLANTA" - 
            if data[0] == "ATLANTA":
                # Extract the hospital name (item 1)
                source_hospital = data[1]
                # Extract the hospital address (item 3)
                hosp_address = data[3]
                # If the hospital name is not already a key in the directory
                if source_hospital not in directory.keys():
                    # Add a new entry with the hospital name as key and the hospital address as the value
                    directory[source_hospital] = hosp_address
                # If the hospital name is already a key in the directory
                elif source_hopsital in directory.keys():
                    # check if the address previously stored is the same as the address collected now
                    if directory[source_hospital] == hosp_address:
                    # If the address is the same - pass
                        pass
                    # If the address is different - raise a key error
                    else:
                        raise KeyError("This hospital({}) has multiple address entries. Please consult the source website.".format(hospital))

    # Once the directory has been created, look for the input address
    # first, standardize the case of the input to uppercase (to match the HTML input)
    # Loop through the keys in the directory dictionary to find a matching key
    for source_hospital, hosp_address in directory.items():
        # If the hospital name input matches a key in the directory dictionary
        if source_hospital == hospital.upper():
            # assign the value of the directory[source_hospital] to the address variable
            address = directory[source_hospital]
    
    # If address has not been assigned and remains None, raise a KeyError
    if address == None:
        raise KeyError("The address for {} was not found. Please check the input and consult the help documentation.".format(hospital))
    
    # If the address is not None, proceed to return the address
    return address

In [13]:
# testing the get_address function
get_address('Select Specialty Hospital - Northeast Atlanta')

'1821 CLIFTON ROAD NE'

In [14]:
# Testing the docstring 
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose = True)

Finding tests in NoName
Trying:
    get_address('Select Specialty Hospital - Northeast Atlanta')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Southwest Hospital and Medical Center')
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [21]:
# define the function with the specifications above
def process_people(file):
    '''(file path) -> list
    This function takes a single file path as input and creates a dictionary, where each entry is formatted as follows:
    { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}
    
    This function relies on three previously created functions: priest(), find_hospital(), and get_address(). See documentation for each function for details.
    This function assumes the file name will have all the fields necessary for calculating the PRIEST score in the order listed above/in the priest() documentation,
    as well as a patient identifier. 
    '''
    # Intialize a dictionary for the result
    people_dict = {}
    
    # import the csv module
    import csv
    
    # Call the file and open it as f
    with open(file) as f:
        # read the file
        reader = csv.reader(f, delimiter = "|")
        # Process each line in the file to: 
        for line in reader:
            # extract the following and save as variables: patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, and status
            patient_number = line[0]
            # to move beyond the header row, check that the patient_number is not "patient"
            if patient_number != "patient":
                sex = line[1]
                age = int(line[2])
                breath = float(line[3])
                o2sat = float(line[4])
                heart = float(line[5])
                systolic = float(line[6])
                temp = float(line[7])
                alertness = line[8]
                inspired = line[9]
                status = line[10]

                # call the priest() function and provide the appropriate inputs, saving the risk score to a variable
                risk = priest(sex, int(age), int(breath), float(o2sat), float(heart), float(systolic), float(temp), alertness, inspired, status)

                # Using the risk score calculated by priest(), the sex, and the age, find the appropriate hospital by calling find_hospital, 
                # saving the hospital name as a variable
                hospital = find_hospital(age, sex, risk)

                # Given the appropriate hospital, call get_address() to retrieve the hospital address
                address = get_address(hospital)

                # If the patient number is not already in the list (people_list):
                if patient_number not in people_dict:
                    # Create a new dictionary entry with the following format:
                    # { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}
                    people_dict[patient_number] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address]
                # If the patient number is already in the dictionary:
                elif patient_number in people_dict:
                   # raise a ValueError
                    raise ValueError("There is a duplicate patient number{} in the file. Please review the source file and correct duplications prior to processing.".format(patient_number))
    
    # Return the finished dictionary
    return people_dict

In [16]:
# Testing the process_people function
process_people("/data/people.psv")

{'E9559': ['FEMALE',
  '40',
  '24',
  '0.96',
  '105',
  '115',
  '34.9',
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  '51',
  '19',
  '0.99',
  '90',
  '91',
  '38.1',
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  '40',
  '29',
  '0.96',
  '105',
  '95',
  '38.1',
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  '66',
  '19',
  '0.96',
  '135',
  '115',
  '38.1',
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'Wesley Woods Geriatric Hospital',
  '1821 CLIFTON ROAD, NE'],
 'E8661': ['MALE',
  '18',
  '22',
  '0.95',
  '105',
  '115',
  '36.3',
  'Confused or Not Alert',
  'air',
  'unrestricted normal activit

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [17]:
# import json module
import json

# read in the people_results.json file
with open("/data/people_results.json") as answers:
    # Create a variable for the data in the people_results file
    provided_answers = json.load(answers)

In [22]:
# Create the output from process_people and save to a variable
midterm_results = process_people("/data/people.psv")

In [23]:
if provided_answers == midterm_results:
    print("Congratulations! You completed the midterm.")

else:
    print("Hm, not quite right yet.")
    # Verify that the midterm results are the provided answers
    for key, value in midterm_results.items():
        if provided_answers[key] != value:
            print("""The answers do not match for the following patient number {}.
            Your answer was {}.
            The provided answer was {}.""".format(key, value, provided_answers[key]))

Congratulations! You completed the midterm.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [20]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main 946467c] Submitting the midterm
 1 file changed, 308 insertions(+), 3 deletions(-)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 3.37 KiB | 1.69 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:ecdmdphd/hds5210-2022.git
   9227777..946467c  main -> main
